In [2]:
# from matplotlib.pyplot import table
import pandas as pd
from sqlalchemy import create_engine
import os
import psycopg2
from sqlalchemy import create_engine
import datetime
import dateutil
from google_play_scraper import Sort, reviews
from tqdm.auto import tqdm

In [3]:
class DatabaseConnection:
    def __init__(self):
        connection_info = {
            "host": os.environ.get("PG_HOST", "localhost"),
            "port": os.environ.get("PG_PORT", 5432),
            "db": os.environ.get("PG_DATABASE", "playstore"),
            "user": os.environ.get("PG_USER", "postgres"),
            "password": os.environ.get("PG_PASS", "postgres"),
        }

        # Sql engine
        self.db_engine = create_engine(
            "postgresql+psycopg2://{}:{}@{}:{}/{}".format(
                connection_info["user"],
                connection_info["password"],
                connection_info["host"],
                connection_info["port"],
                connection_info["db"],
            )
        )
        # psql engine
        self.db_conn = psycopg2.connect(
            host=connection_info["host"],
            port=connection_info["port"],
            database=connection_info["db"],
            user=connection_info["user"],
            password=connection_info["password"],
        )

In [8]:
# Get data
def get_data(dttm):
    connection_info = {
        "host": os.environ.get("PG_HOST", "localhost"),
        "port": os.environ.get("PG_PORT", 5432),
        "db": os.environ.get("PG_DATABASE", "playstore"),
        "user": os.environ.get("PG_USER", "postgres"),
        "password": os.environ.get("PG_PASS", "postgres"),
    }

    # Sql engine
    db_engine = create_engine(
        "postgresql+psycopg2://{}:{}@{}:{}/{}".format(
            connection_info["user"],
            connection_info["password"],
            connection_info["host"],
            connection_info["port"],
            connection_info["db"],
        )
    )

    query = f"""
        SELECT "review"."reviewId"
        , "review"."apps"
        , "review"."score"
        , "review"."at"
        , "review"."content"
        , "review"."repliedAt"
        , "sentiment"."clean_text"
        , "sentiment"."sentiment"
        FROM review
        LEFT JOIN sentiment
        ON ("review"."reviewId"="sentiment"."reviewId")
        WHERE "sentiment"."sentiment" is not null
        AND "review"."at" > '{dttm}'
        AND "review"."apps" in ('id.co.cimbniaga.mobile.android','net.myinfosys.PermataMobileX','id.bmri.livin')
        ;
        """
    df = pd.read_sql(query, db_engine)
    df["repliedDurationHrs"] = (df["repliedAt"] - df["at"]).apply(
        lambda x: round(x.seconds / 3600, 2)
    )

    return df

In [12]:
df = get_data('2022-01-01')
df

,reviewId,apps,score,at,content,repliedAt,clean_text,sentiment,repliedDurationHrs
0,8ec5fab4-6c23-4524-ae85-cbb2ba578e3f,id.bmri.livin,5,2022-07-14 10:13:39,ok,2022-08-12 23:54:02,oke,positive,13.67
1,190052db-d4d9-498c-a2e9-06d18c038587,id.bmri.livin,5,2022-07-03 14:46:03,Bgs.,2022-07-04 07:46:45,Bgs,negative,17.01
2,cc36c85f-8e59-42da-bb60-a68cdd7a8cfc,id.bmri.livin,1,2022-09-24 21:08:40,Gagal terus verifikasinya..habis dah pulsa bua...,2022-09-27 16:26:09,Gagal terus verifikasinya habis deh pulsa buat...,negative,19.29
3,9899d237-f581-4f92-978c-af00c9539e07,id.bmri.livin,1,2022-09-24 23:10:02,"Pendaftaran ribet , nga ada menu buat loogin k...",2022-09-27 23:00:34,Pendaftaran ribet enggak ada menu buat loogin...,negative,23.84
4,275618f9-cac9-4750-9373-ca86722cb734,id.bmri.livin,5,2022-09-21 10:02:33,Mantaap,2022-09-21 11:22:01,Mantaap,positive,1.32
...,...,...,...,...,...,...,...,...,...
208147,3740d0a3-29eb-4d1e-b788-6e5403cb4779,id.bmri.livin,1,2022-02-25 22:10:12,Sangat mengecewakan. Awal download lancar2 aja...,2022-03-01 15:52:30,Sangat mengecewakan Awal unduh lancar saja Mak...,negative,17.70
208148,09d16db1-b7c4-41f4-95e6-8615a9e7a5bc,id.bmri.livin,1,2022-02-25 22:10:05,Tidak lebih baik dari living yg biru malah tam...,2022-03-01 15:57:13,Tidak lebih baik dari living yang biru malah t...,negative,17.79
208149,acc0a6e0-df5c-4a17-862d-8c41fa1f2706,id.bmri.livin,1,2022-02-25 21:55:50,Tidak bsa di buka selalu bacaan coba beberapa ...,2022-03-01 19:24:41,Tidak bisa di buka selalu bacaan coba beberapa...,negative,21.48
208150,33eb1b1f-c786-4984-9bd6-2a372f9f2ad3,id.bmri.livin,5,2022-02-25 22:12:50,kok trobel ya,2022-03-01 16:13:13,kok trobel ya,negative,18.01


In [11]:
df.to_csv("payload.csv",index=False)

In [4]:
# Create function get PS Review based on date
app_id = "id.co.bri.brimo"
app_id = "net.myinfosys.PermataMobileX"
conn = DatabaseConnection().db_conn
MAX_COUNT_EACH_FETCH = 200

In [5]:
# helper.status


def get_last_date_db(app_id, conn=None):
    # create query
    date_query = f"""
        SELECT MAX(at)
        FROM review
        WHERE apps = '{app_id}'
        """

    # get dttm from db
    curr = conn.cursor()

    # execute query
    curr.execute(date_query)
    t = curr.fetchall()[0][0]
    if not isinstance(t, datetime.date) and t:
        t = dateutil.parser.parse(t)
    if not t:
        t = dateutil.parser.parse("2023-02-01")
    return t

In [6]:
continuation_token = None
result = []
condition = False
last_date = get_last_date_db(app_id, conn)

In [ ]:
def get_review_by_last_date(app_id, last_date) -> pd.DataFrame:
    results = []
    continuation_token = None
    is_date_match = False

    pbar = tqdm()
    while is_date_match is False:
        try:
            result, continuation_token = reviews(
                app_id,
                count=MAX_COUNT_EACH_FETCH,
                continuation_token=continuation_token,
                lang="id",  # defaults to 'en'
                country="id",  # defaults to 'us'
                sort=Sort.NEWEST,  # defaults to Sort.MOST_RELEVANT
                filter_score_with=None,  # defaults to None(means all score)
            )
            results += result
            results_date = results[-1]["at"]
            pbar.update(len(results))
        except:
            continue

        if len(results) % 10000 == 0:
            print(results_date)

        if results[-1]["at"] <= last_date:
            is_date_match = True

    df = pd.DataFrame(result)
    return df

In [ ]:
last_date = dateutil.parser.parse("2023-02-01")

In [ ]:
df = get_review_by_last_date(app_id, last_date)

In [ ]:
df

In [ ]:
results = []
continuation_token = None
is_date_match = False

pbar = tqdm()
while is_date_match is False:
    result, continuation_token = reviews(
        app_id,
        count=MAX_COUNT_EACH_FETCH,
        continuation_token=continuation_token,
        lang="id",  # defaults to 'en'
        country="id",  # defaults to 'us'
        sort=Sort.NEWEST,  # defaults to Sort.MOST_RELEVANT
        filter_score_with=None,  # defaults to None(means all score)
    )
    results += result
    results_date = results[-1]["at"]
    pbar.update(len(results))

    if len(results) % 1000 == 0:
        print(results_date)

    if results[-1]["at"] <= last_date:
        is_date_match = True

In [ ]:
# -1 adalah tanggal yang paling lama , idx 1 adalah tanggal terbaru
results[-1]["at"] <= last_date

# Get data from postgress

In [6]:
dttm = "2022-06-01"
query = f"""
    SELECT "review"."reviewId"
    , "review"."apps"
    , "review"."score"
    , "review"."at"
    , "review"."content"
    , "review"."repliedAt"
    , "sentiment"."clean_text"
    , "sentiment"."sentiment"
    FROM review
    LEFT JOIN sentiment
    ON ("review"."reviewId"="sentiment"."reviewId")
    WHERE "sentiment"."sentiment" is not null
    AND "review"."apps" in ('net.myinfosys.PermataMobileX', 'id.bmri.livin', 'id.co.cimbniaga.mobile.android')
    AND "review"."at" > '{dttm}';
    """

In [7]:
df = pd.read_sql(query, DatabaseConnection().db_conn)


In [8]:
df = df[['apps', 'score', 'content', 'clean_text', 'sentiment']]

In [11]:
df.groupby('apps').size()

apps
id.bmri.livin                     82198
id.co.cimbniaga.mobile.android    13940
net.myinfosys.PermataMobileX       8787
dtype: int64

In [13]:
df.to_csv("3bank.csv", sep=',', index=False)

In [9]:
# Group your data by category
grouped = df.groupby('sentiment')

# Randomly sample 1000 data points from each group
sampled = grouped.apply(lambda x: x.sample(n=3000))

# Concatenate the randomly sampled data from each group into a new dataframe
result = pd.concat([sampled], axis=0)

In [10]:
result.to_csv("raw_data.csv",sep=';')